In [ ]:
# GAN

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot = True)

C:\Users\ICT01_06\Anaconda3\envs\tf_test22\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_06\Anaconda3\envs\tf_test22\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_06\Anaconda3\envs\tf_test22\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_06\Anaconda3\envs\tf_test22\lib\site-

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [3]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden],stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input],stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden],stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [4]:
# 생성기와 판별기
# cost function : 확률적 함수
# MSE , Enthropy 분류
# KL-Divergence : GAN, VAE - 분포의 차를 확인하는 함수
# 생성기 -> noise 가 input 데이터, cost function : KL- Divergence
# 
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1) # 
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2) + D_b2)
    return output
def get_noise(batch_size, n_noise):
    return np.random.normal(size = (batch_size,n_noise))

In [5]:
G = generator(Z) # 노이즈 -> 이미지 생성
D_gene = discriminator(G) # 이미지를 판별 - 분포를 확인한다
# log 확률값 -> 정보량
# 확률이 높아지면 정보량이 작아지고
# 확률이 낮아지면 정보량이 커짐 tfidf의 논리를 고려할것
# 가장 적합한 분포 -> 높은 것이 유리
D_real = discriminator(X)
# 마이너스를 붙이면 최대화 하기하는것
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)) # 최대우도 추정법 -적합한 분포
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
# 마이너스를 붙이면 최대화 하기하는것
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
# 역전파 변수를 지정
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [6]:
%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

In [11]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X:batch_xs, Z:noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    print('Epoch:', '%04d' % epoch, ' D loss: {:.4}'.format(loss_val_D),
          'G loss :{:.4}'.format(loss_val_G))
    if epoch ==0 or (epoch +1) % 10 == 0:
        sample_size =10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
            plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
            plt.close(fig)
print('최적화완료')    

Epoch: 0000  D loss: -0.07162 G loss :-3.417
Epoch: 0001  D loss: -0.4231 G loss :-1.664
Epoch: 0002  D loss: -0.2935 G loss :-2.175
Epoch: 0003  D loss: -0.3198 G loss :-2.316
Epoch: 0004  D loss: -0.3284 G loss :-2.668
Epoch: 0005  D loss: -0.1633 G loss :-3.273
Epoch: 0006  D loss: -0.2076 G loss :-3.44
Epoch: 0007  D loss: -0.2015 G loss :-3.65
Epoch: 0008  D loss: -0.1456 G loss :-3.5
Epoch: 0009  D loss: -0.222 G loss :-2.922
Epoch: 0010  D loss: -0.331 G loss :-2.903
Epoch: 0011  D loss: -0.3212 G loss :-2.821
Epoch: 0012  D loss: -0.3999 G loss :-2.476
Epoch: 0013  D loss: -0.2694 G loss :-2.82
Epoch: 0014  D loss: -0.3285 G loss :-2.477
Epoch: 0015  D loss: -0.309 G loss :-2.354
Epoch: 0016  D loss: -0.5354 G loss :-2.527
Epoch: 0017  D loss: -0.3869 G loss :-2.659
Epoch: 0018  D loss: -0.399 G loss :-2.607
Epoch: 0019  D loss: -0.3516 G loss :-2.693
Epoch: 0020  D loss: -0.3731 G loss :-2.715
Epoch: 0021  D loss: -0.5221 G loss :-2.363
Epoch: 0022  D loss: -0.4307 G loss :-2.

<Figure size 432x288 with 0 Axes>

In [10]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X:batch_xs,Z:noise})
        # 이미지를 생성해줌 128 * 128 로 입력
        _, loss_val_D = sess.run([train_G, loss_G], feed_dict={Z:noise})
    print('Epoch %04d'%epoch,'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
    if epoch == 0 or (epoch+1) % 10 ==0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise) # 10 * 128
        samples = ses.run(G, feed_dict={Z: noise})
        fig, ax = plt.subplot(1, sample_size, figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()# 128 -> 784 이미지 사이즈
            ax[i].imshow(np.reshape(samples[i], (28,28)))
            plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
            plt.close(fig)
print('최적화 완료')

ValueError: Precision not allowed in integer format specifier